In [4]:
!pip install transformers[sentencepiece] datasets sacrebleu rouge_score py7zr -q
!pip install peft

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 2.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.9/67.9 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 48.4 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.4/96.4 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.3/141.3 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 412.9/412.9 kB 31.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cesium 0.12.4 requires numpy<3.0,>=2.0, but you have numpy 1.

In [5]:
import os
import torch
from datasets import load_dataset,DatasetDict
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, Trainer, TrainingArguments
from peft import LoraConfig, get_peft_model, TaskType
from transformeimport evaluate
import evaluate

# torch.manual_seed(42)

# dataset = load_dataset("SurAyush/News_Summary_Dataset")
# dataset = dataset.shuffle(seed=42)
# dataset

2025-06-02 04:26:17.043439: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748838377.238141      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748838377.296273      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [6]:
from datasets import load_dataset, concatenate_datasets, DatasetDict

dataset = load_dataset("SurAyush/News_Summary_Dataset")["train"]
data_1 = load_dataset("gopalkalpande/bbc-news-summary")["train"]

data_1 = data_1.remove_columns(["File_path"])
data_1 = data_1.rename_columns({"Articles": "article", "Summaries": "summary"})

combined_dataset = concatenate_datasets([data_1, dataset])

final_dataset = DatasetDict({"train": combined_dataset})

print(final_dataset)


README.md:   0%|          | 0.00/1.22k [00:00<?, ?B/s]

news_summaries.csv:   0%|          | 0.00/7.31M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/2224 [00:00<?, ? examples/s]

README.md:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

bbc-news-summary.csv:   0%|          | 0.00/7.32M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/2224 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['article', 'summary'],
        num_rows: 4448
    })
})


In [7]:
split_dataset =  final_dataset["train"].train_test_split(test_size=0.1, seed=42)
print("Dataset after split:", split_dataset)

Dataset after split: DatasetDict({
    train: Dataset({
        features: ['article', 'summary'],
        num_rows: 4003
    })
    test: Dataset({
        features: ['article', 'summary'],
        num_rows: 445
    })
})


In [8]:
model_name = "t5-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [9]:
def preprocess_function(examples):
    inputs = ["summarize: " + doc for doc in examples["article"]]
    model_inputs = tokenizer(
        inputs,
        max_length=512,
        truncation=True,
        padding="max_length",
    )
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            examples["summary"],
            max_length=256,
            truncation=True,
            padding="max_length",
        )
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [11]:
tokenized_datasets = split_dataset.map(preprocess_function, batched=True)
tokenized_datasets = tokenized_datasets.remove_columns(["article", "summary"])
print("Tokenized datasets:", tokenized_datasets)

Map:   0%|          | 0/4003 [00:00<?, ? examples/s]

Tokenized datasets: DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 4003
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 445
    })
})


In [12]:
train_dataset = tokenized_datasets["train"].select(range(4000))
eval_dataset = tokenized_datasets["test"].select(range(410)) 

In [13]:
print("Train dataset:", train_dataset) 
print("Validation dataset:", eval_dataset) 

Train dataset: Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 4000
})
Validation dataset: Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 410
})


In [15]:
lora_config = LoraConfig(
    task_type=TaskType.SEQ_2_SEQ_LM,
    r=8,  
    lora_alpha=32,
    lora_dropout=0.1,
    target_modules=["q", "v"],  
)

# Apply LoRA to the model
model = get_peft_model(model, lora_config)
model.print_trainable_parameters() 

trainable params: 884,736 || all params: 223,788,288 || trainable%: 0.3953


In [16]:
# Define training arguments
training_args = TrainingArguments(
    output_dir="/kaggle/working/lora-t5-news-summarization",eval_strategy="epoch",learning_rate=1e-3,per_device_train_batch_size=5,per_device_eval_batch_size=5,
    num_train_epochs=3,weight_decay=0.01,
    save_strategy="epoch",
    save_total_limit=2,
    load_best_model_at_end=True,fp16=True, 
    logging_dir="/kaggle/working/logs",logging_steps=100,
    report_to='none'
)


In [ ]:
rouge = evaluate.load("rouge")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    
    # Replace -100 in the labels as we can't decode them
    labels = [[(l if l != -100 else tokenizer.pad_token_id) for l in label] for label in labels]
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    return {k: round(v, 4) for k, v in result.items()}


In [17]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics
)

trainer.train()  

No label_names provided for model class `PeftModelForSeq2SeqLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,0.456200,0.395610
2,0.462100,0.373885
3,0.431800,0.367535


/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


TrainOutput(global_step=1200, training_loss=0.5467392158508301, metrics={'train_runtime': 1367.5798, 'train_samples_per_second': 8.775, 'train_steps_per_second': 0.877, 'total_flos': 7340109594624000.0, 'train_loss': 0.5467392158508301, 'epoch': 3.0})

In [26]:
model.save_pretrained("/kaggle/working/lora-t5-news-final")
tokenizer.save_pretrained("/kaggle/working/lora-t5-news-final") 

('/kaggle/working/lora-t5-news-final/tokenizer_config.json',
 '/kaggle/working/lora-t5-news-final/special_tokens_map.json',
 '/kaggle/working/lora-t5-news-final/spiece.model',
 '/kaggle/working/lora-t5-news-final/added_tokens.json',
 '/kaggle/working/lora-t5-news-final/tokenizer.json')

In [ ]:
metrics = trainer.evaluate()
print("Evaluation metrics:", metrics)

In [28]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [29]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from peft import PeftModel, PeftConfig

In [30]:
model_path = "/kaggle/working/lora-t5-news-final"
tokenizer = AutoTokenizer.from_pretrained(model_path)
peft_config = PeftConfig.from_pretrained(model_path)
base_model = AutoModelForSeq2SeqLM.from_pretrained(peft_config.base_model_name_or_path)
model = PeftModel.from_pretrained(base_model, model_path)

In [31]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

PeftModelForSeq2SeqLM(
  (base_model): LoraModel(
    (model): T5ForConditionalGeneration(
      (shared): Embedding(32128, 768)
      (encoder): T5Stack(
        (embed_tokens): Embedding(32128, 768)
        (block): ModuleList(
          (0): T5Block(
            (layer): ModuleList(
              (0): T5LayerSelfAttention(
                (SelfAttention): T5Attention(
                  (q): lora.Linear(
                    (base_layer): Linear(in_features=768, out_features=768, bias=False)
                    (lora_dropout): ModuleDict(
                      (default): Dropout(p=0.1, inplace=False)
                    )
                    (lora_A): ModuleDict(
                      (default): Linear(in_features=768, out_features=8, bias=False)
                    )
                    (lora_B): ModuleDict(
                      (default): Linear(in_features=8, out_features=768, bias=False)
                    )
                    (lora_embedding_A): ParameterDict()
               

In [46]:
text = """
BJP Returns to Power in Delhi After 27 Years: Rekha Gupta Becomes Chief Minister

In a landmark political shift, the Bharatiya Janata Party (BJP) has reclaimed power in the Delhi Legislative Assembly after a 27-year hiatus. The 2025 elections, held on February 5, culminated in a decisive victory for the BJP, which secured 48 out of 70 seats, surpassing the majority threshold of 36 seats. This triumph ended the Aam Aadmi Party's (AAP) decade-long governance in the national capital. 
India TV News
+1
India Today
+1

Election Outcomes
BJP: 48 seats

AAP: 22 seats

Congress: 0 seats
India TV News
+3
India Today
+3
Wikipedia
+3
@mathrubhumi
+5
The Times of India
+5
Wikipedia
+5
Business Standard

The BJP's vote share stood at 47.15%, marking a significant increase from the 38.51% it garnered in the 2020 elections. Conversely, AAP's vote share declined to 43.57% from its previous 53.57%. 
India TV News
+1
Wikipedia
+1

Leadership Changes
Following the BJP's victory, Rekha Gupta was appointed as the Chief Minister of Delhi. Gupta, who won from the Shalimar Bagh constituency, became the first woman to hold this position in the city. Her leadership marks a new chapter in Delhi's political landscape. 
Wikipedia

Notable Defeats
The election results were particularly significant for the AAP, as several of its prominent leaders faced defeats:
India TV News

Arvind Kejriwal: The former Chief Minister lost the New Delhi seat to BJP's Parvesh Verma.

Manish Sisodia: The former Deputy Chief Minister was defeated in the Jangpura constituency by BJP's Tarvinder Singh Marwah.

Saurabh Bharadwaj: The outgoing Health Minister lost the Greater Kailash seat to BJP's Shikha Roy.
The Times of India
+1
India TV News
+1
India Today
+2
India TV News
+2
Business Standard
+2

Despite these setbacks, some AAP leaders managed to retain their seats:

Atishi: Won from the Kalkaji constituency.

Gopal Rai: Secured the Babarpur seat.

Imran Hussain: Retained the Ballimaran constituency.
India Today

Reactions and Future Outlook
Prime Minister Narendra Modi hailed the BJP's victory as a mandate for development and good governance. He expressed gratitude to the people of Delhi for their support and emphasized the party's commitment to fulfilling its promises. 
Business Standard
+1
WIONews
+1
WIONews

The AAP, on the other hand, acknowledged the electoral setback and pledged to introspect and rebuild. Party leaders emphasized their continued dedication to public service and vowed to play a constructive role in the opposition.

As the BJP assumes control of Delhi's administration, the focus is expected to shift towards implementing its development agenda, enhancing infrastructure, and addressing key issues such as pollution and public transportation. The political dynamics in the capital are poised for significant changes under the new leadership.
"""

In [48]:
inputs = tokenizer("summarize: " + text, return_tensors="pt", max_length=1024, truncation=True).to(device)

outputs = model.generate(**inputs, max_length=300, min_length=80 ,num_beams=4 ,early_stopping=True  ,length_penalty=1.0, no_repeat_ngram_size=3)
summary = tokenizer.decode(outputs[0], skip_special_tokens=True)

print("summary:", summary)


summary: Indian TV News +1 India Today +1 Election Outcomes BJP: 48 seats AAP: 22 seats Congress: 0 seats India TV News +3 India Today +3 Wikipedia +3 @mathrubhumi +5 The Times of India +5 Wikipedia +5 Business Standard Despite these setbacks, some AAP leaders managed to retain their seats: Atishi: Won from the Kalkaji constituency.Indead TV News: +1 Wikipedia +1 Leadership Changes Following the BJP's victory, Rekha Gupta was appointed as the Chief Minister of Delhi.Manish Sisodia: The former Deputy Chief Minister was defeated in the Jangpura constituency, became the first woman to hold this position in the city.In a landmark political shift, the Bharatiya Janata Party (BJP) has reclaimed power in the Delhi Legislative Assembly after a 27-year hiatus.
